In [14]:
import dask.dataframe as dd

# Carregar o calendário com a coluna 'date' como datetime
calendar = dd.read_csv('../data/daily_calendar_with_events.csv', parse_dates=['date'])

In [15]:
def add_yearweek(partition):
    isocal = partition['date'].dt.isocalendar()
    # Formatar 'yearweek' como string com zero padding para a semana
    partition['yearweek'] = isocal.year.astype(str) + isocal.week.astype(str).str.zfill(2)
    return partition

# Aplicar a função a cada partição do DataFrame
calendar = calendar.map_partitions(add_yearweek, meta=calendar.assign(yearweek='object'))

# Verificar as primeiras linhas
print(calendar.head())

        date    weekday  weekday_int    d event yearweek
0 2011-01-29   Saturday            1  d_1   NaN   201104
1 2011-01-30     Sunday            2  d_2   NaN   201104
2 2011-01-31     Monday            3  d_3   NaN   201105
3 2011-02-01    Tuesday            4  d_4   NaN   201105
4 2011-02-02  Wednesday            5  d_5   NaN   201105


In [16]:
# Verificar os tipos das colunas
print(calendar.dtypes)


date            datetime64[ns]
weekday        string[pyarrow]
weekday_int              int64
d              string[pyarrow]
event          string[pyarrow]
yearweek                object
dtype: object


In [17]:
# Salvar como múltiplos arquivos Parquet
calendar.to_parquet('../data/calendar_with_yearweek_parquet/', write_index=False)